In [ ]:
import pandas as pd
import numpy as np
import re # Регулярки
from math import gcd, isclose
import sympy as sp # Библиотека символьных вычеслений, мат.задачи

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv(r'/content/Bias_data_1.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1272 non-null   object 
 1   Unnamed: 1         1277 non-null   object 
 2   Unnamed: 2         1298 non-null   int64  
 3   id                 1298 non-null   object 
 4   link               1298 non-null   object 
 5   space_group_core   1276 non-null   float64
 6   space_group_shell  1259 non-null   float64
 7   core1              1298 non-null   object 
 8   shell1             1298 non-null   object 
 9   h_range_max_koe    1245 non-null   float64
 10  chemical formula   1298 non-null   object 
 11  Tb                 988 non-null    float64
 12  TN                 1109 non-null   float64
 13  Tc                 1048 non-null   float64
 14  c1                 1298 non-null   object 
 15  c2                 1284 non-null   object 
 16  x                  1234 

In [ ]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'id', 'link',
       'space_group_core', 'space_group_shell', 'core1', 'shell1',
       'h_range_max_koe', 'chemical formula', 'Tb', 'TN', 'Tc', 'c1', 'c2',
       'x', 'y', 'z', 'shape', 'temperature_k', 'sat_em_g', 'coer_oe',
       'mr (emu/g)', 'exc_bias_oe', 'exc_dir', 'ver_shift_emu_g', 'ver_s_dir',
       'fc_field_t', 'Cas rn (core_1)', 'Cas rn (core_2)'],
      dtype='object')

In [ ]:
columns = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Cas rn (core_1)', 'Cas rn (core_2)']
df = df.drop(columns = columns)
df

,id,link,space_group_core,space_group_shell,core1,shell1,h_range_max_koe,chemical formula,Tb,TN,...,shape,temperature_k,sat_em_g,coer_oe,mr (emu/g),exc_bias_oe,exc_dir,ver_shift_emu_g,ver_s_dir,fc_field_t
0,1,10.1063/1.4771993,186.0,227.0,Zn1O1,Co1Fe2O4,50.0,ZnO/CoFe2O4,300.0,NaN,...,1.0,5.0,NaN,27800.000,NaN,1132.000,0.0,0.000,1.0,5.0
1,2,10.1063/1.4771993,225.0,227.0,Co1O1,Co1Fe2O4,50.0,CoO/CoFe2O4,276.0,293.0,...,1.0,5.0,NaN,7800.000,NaN,406.000,0.0,0.000,1.0,5.0
2,3,10.1016/j.jmmm.2022.170252,227.0,0.0,Mn0.50Zn0.50Fe2O4,0,10.0,Mn0.5Zn0.5Fe2O4,150.0,NaN,...,1.0,300.0,2.290,24.800,0.0078,51.340,1.0,0.000,1.0,0.0
3,4,10.1016/j.jmmm.2022.170252,227.0,0.0,Mn0.50Zn0.50Fe2O4,0,10.0,Mn0.5Zn0.5Fe2O4,150.0,NaN,...,1.0,300.0,2.430,90.200,0.0218,108.180,1.0,0.000,1.0,0.0
4,5,10.1016/j.jmmm.2022.170252,227.0,0.0,Mn0.50Zn0.50Fe2O4,0,10.0,Mn0.5Zn0.5Fe2O4,150.0,NaN,...,1.0,300.0,16.150,40.070,0.3498,8.920,1.0,0.000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,val 28,10.1109/TMAG.2024.3418626,161.0,0.0,BiFeO3,0,70.0,BiFeO3-SG,200.0,640.0,...,1.0,2.0,NaN,1923.000,NaN,1180.000,0.0,NaN,1.0,7.0
1294,val 29,10.1109/TMAG.2024.3418626,161.0,0.0,BiFeO3,0,70.0,BiFeO3-SG,200.0,640.0,...,1.0,2.0,NaN,450.000,NaN,126.000,0.0,NaN,1.0,7.0
1295,val 30,10.1016/j.ceramint.2019.02.028,15.0,225.0,NiCo2O4,NiO,30.0,NiCo2O4/NiO,75.0,490.0,...,1.0,10.0,0.845,619.815,0.1070,432.315,0.0,0.081,1.0,0.3
1296,val 31,10.1016/j.ceramint.2019.02.028,15.0,225.0,NiCo2O4,NiO,30.0,NiCo2O4/NiO,75.0,490.0,...,1.0,30.0,0.783,289.735,0.6500,289.735,0.0,0.650,1.0,0.3


In [ ]:
df['x'] = df['x'].astype('float64')
df['y'] = df['y'].astype('float64')
df['z'] = df['z'].astype('float64')

In [ ]:
# Преобразование типа данных в числовой
df['shape'] = df['shape'].astype('float64')
df['space_group_core'] = df['space_group_core'].astype('float64')
df['space_group_shell'] = df['space_group_shell'].astype('float64')
#df['crystal_structure'] = df['crystal_structure'].astype('float64')

# Заполнение пропущенных значений в категориальных признаках модой
df['shape'] = df['shape'].fillna(df['shape'].mode()[0])
df['space_group_core'] = df['space_group_core'].fillna(df['space_group_core'].mode()[0])
df['space_group_shell'] = df['space_group_shell'].fillna(df['space_group_shell'].mode()[0])
#df['crystal_structure'] = df['crystal_structure'].fillna(df['crystal_structure'].mode()[0])

# Преобразование типа данных в категориальный
df['shape'] = df['shape'].astype('category')
df['space_group_core'] = df['space_group_core'].astype('category')
df['space_group_shell'] = df['space_group_shell'].astype('category')
#df['crystal_structure'] = df['crystal_structure'].astype('category')

In [ ]:
df['exc_dir'] = df['exc_dir'].astype('category')
df['ver_s_dir'] = df['ver_s_dir'].astype('category')

In [ ]:
df.columns

Index(['id', 'link', 'space_group_core', 'space_group_shell', 'core1',
       'shell1', 'h_range_max_koe', 'chemical formula', 'Tb', 'TN', 'Tc', 'c1',
       'c2', 'x', 'y', 'z', 'shape', 'temperature_k', 'sat_em_g', 'coer_oe',
       'mr (emu/g)', 'exc_bias_oe', 'exc_dir', 'ver_shift_emu_g', 'ver_s_dir',
       'fc_field_t'],
      dtype='object')

In [ ]:
core, shel, all = df['c1'], df['c2'], df['chemical formula']

In [ ]:
df  = df.drop(columns =['link', 'core1', 'shell1', 'chemical formula',
                        'c1', 'c2'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 1298 non-null   object  
 1   space_group_core   1298 non-null   category
 2   space_group_shell  1298 non-null   category
 3   h_range_max_koe    1245 non-null   float64 
 4   Tb                 988 non-null    float64 
 5   TN                 1109 non-null   float64 
 6   Tc                 1048 non-null   float64 
 7   x                  1234 non-null   float64 
 8   y                  1212 non-null   float64 
 9   z                  1209 non-null   float64 
 10  shape              1298 non-null   category
 11  temperature_k      1265 non-null   float64 
 12  sat_em_g           953 non-null    float64 
 13  coer_oe            1252 non-null   float64 
 14  mr (emu/g)         1024 non-null   float64 
 15  exc_bias_oe        1267 non-null   float64 
 16  exc_di

In [ ]:
df.columns

Index(['id', 'space_group_core', 'space_group_shell', 'h_range_max_koe', 'Tb',
       'TN', 'Tc', 'x', 'y', 'z', 'shape', 'temperature_k', 'sat_em_g',
       'coer_oe', 'mr (emu/g)', 'exc_bias_oe', 'exc_dir', 'ver_shift_emu_g',
       'ver_s_dir', 'fc_field_t'],
      dtype='object')

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

In [ ]:
df_for_kNN = df.drop(columns=['id'])

# Нормализация данных
scaler = StandardScaler()
normalized_data = scaler.fit_transform(df_for_kNN)

# Заполним пропущенные значения в датасете с помощью kNN
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(normalized_data)
df_imputed = pd.DataFrame(imputed_data, columns=df_for_kNN.columns)

# Объединим заполненные данные с исходным датасетом
df_filled = pd.concat([df[['id']], df_imputed], axis=1)

# Выведем информацию о пропущенных значениях после заполнения
print(f'Процент пропущенных значений после заполнения: \n{df_filled.isna().sum() * 100 / len(df_filled)}')

Процент пропущенных значений после заполнения: 
id                   0.0
space_group_core     0.0
space_group_shell    0.0
h_range_max_koe      0.0
Tb                   0.0
TN                   0.0
Tc                   0.0
x                    0.0
y                    0.0
z                    0.0
shape                0.0
temperature_k        0.0
sat_em_g             0.0
coer_oe              0.0
mr (emu/g)           0.0
exc_bias_oe          0.0
exc_dir              0.0
ver_shift_emu_g      0.0
ver_s_dir            0.0
fc_field_t           0.0
dtype: float64


In [ ]:
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1298 non-null   object 
 1   space_group_core   1298 non-null   float64
 2   space_group_shell  1298 non-null   float64
 3   h_range_max_koe    1298 non-null   float64
 4   Tb                 1298 non-null   float64
 5   TN                 1298 non-null   float64
 6   Tc                 1298 non-null   float64
 7   x                  1298 non-null   float64
 8   y                  1298 non-null   float64
 9   z                  1298 non-null   float64
 10  shape              1298 non-null   float64
 11  temperature_k      1298 non-null   float64
 12  sat_em_g           1298 non-null   float64
 13  coer_oe            1298 non-null   float64
 14  mr (emu/g)         1298 non-null   float64
 15  exc_bias_oe        1298 non-null   float64
 16  exc_dir            1298 

In [ ]:
# Преобразование обратно в наши значения
org_copy = scaler.inverse_transform(df_filled.iloc[:, 1:]) # Без столбца id
df_recopy = pd.DataFrame(org_copy, columns=df_filled.columns[1:]) # Данные в исходном виде

df_recopy['id'] = id
print(df_recopy['sat_em_g'].unique())

[51.8758 59.734   2.29   ...  0.845   0.783   0.67  ]


In [ ]:
print(df_recopy.info())
print(df_recopy['shape'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   space_group_core   1298 non-null   float64
 1   space_group_shell  1298 non-null   float64
 2   h_range_max_koe    1298 non-null   float64
 3   Tb                 1298 non-null   float64
 4   TN                 1298 non-null   float64
 5   Tc                 1298 non-null   float64
 6   x                  1298 non-null   float64
 7   y                  1298 non-null   float64
 8   z                  1298 non-null   float64
 9   shape              1298 non-null   float64
 10  temperature_k      1298 non-null   float64
 11  sat_em_g           1298 non-null   float64
 12  coer_oe            1298 non-null   float64
 13  mr (emu/g)         1298 non-null   float64
 14  exc_bias_oe        1298 non-null   float64
 15  exc_dir            1298 non-null   float64
 16  ver_shift_emu_g    1298 

In [ ]:
df = df_recopy

In [ ]:
# Добавим колонки для наших значений
df.insert(0, 'area', 0)
df.insert(0, 'volume', 0)
df = df.astype({'volume': float ,'area': float})

In [ ]:
# Создадим словари для подсчёта площади и объёма
dict_area = {
    1: lambda x, y=1, z=1: 4 * 3.14159 * x ** 2,  # Сфера
    2: lambda x, y=1, z=1: (3 ** 2) * 2 * x ** 2,  # Площадь для случая 2
    3: lambda x, y=1, z=1: 6 * x ** 2,  # Куб
    4: lambda x, y=1, z=1: (3 ** (1/2)) * x ** 2,  # Правильный тетраэдр
    5: lambda x, y=1, z=1: (3 ** (1/2)) * x ** 2,  # Повторяющийся случай 4?
    6: lambda x, y=1, z=0: 2 * 3.14159 * (x / 2) * x + 2 * 3.14159 * (x / 2) ** 2,  # Цилиндр
    7: lambda x, y=1, z=1: 6 * x ** 2,  # Куб (или повторение другого случая)
}

dict_volume = {
    1: lambda x, y=1, z=1: (4/3) * 3.14159 * x ** 3,  # Сфера
    2: lambda x, y=1, z=1: (2/3) * (2 ** 2) * x ** 3,  # Объем для случая 2
    3: lambda x, y=1, z=1: x ** 3,  # Куб
    4: lambda x, y=1, z=1: (((2) ** (1/2)) * x ** 3) / 12,  # Правильный тетраэдр
    5: lambda x, y=1, z=1: (((2) ** (1/2)) * x ** 3) / 12,  # Повторяющийся случай 4?
    6: lambda x, y=1, z=0: 3.14159 * (x / 2) ** 2 * y,  # Цилиндр
    7: lambda x, y=1, z=1: x ** 3  # Куб (или повторение другого случая)
}

In [ ]:
# Создаем функцию для вычисления площади
def calculate_area(row):
    shape = float(row['shape'])
    x = row['x']
    y = row.get('y', 0)
    z = row.get('z', 0)

    if shape in dict_area:
        if shape in [1, 2, 3, 4, 5, 6, 7]:
            return dict_area[shape](x, y) if shape in [3, 6] else dict_area[shape](x, z) if shape == 4 else dict_area[shape](x)
    else:
        return 'error'

# Создаем функцию для вычисления объема
def calculate_volume(row):
    shape = float(row['shape'])
    x = row['x']
    y = row.get('y', 0)
    z = row.get('z', 0)

    if shape in dict_volume:
        if shape in [1, 2, 3, 4, 5, 6, 7]:
            return dict_volume[shape](x, y, z) if shape in [3, 6] else dict_volume[shape](x, z) if shape == 4 else dict_volume[shape](x)
    else:
        return 'error'

In [ ]:
df['area'] = df.apply(calculate_area, axis=1)
df['volume'] = df.apply(calculate_volume, axis=1)

In [ ]:
print(df['area'].unique())

[6.88133874e+02 6.15751640e+02 3.63167804e+03 1.52052956e+03
 1.80955584e+03 4.07150064e+03 5.54176476e+03 9.16087644e+03
 1.62860026e+04 1.13097240e+04 2.49415316e+02 4.43405007e+02
 1.03680000e+04 1.96020000e+04 3.93691492e+03 5.22961638e+03
 7.42030992e+03 1.36019537e+04 7.85397500e+03 5.02654400e+05
 4.71238500e+06 5.30928710e+04 1.70553780e+03 4.63246295e+03
 5.57644791e+03 1.36847660e+04 3.40055805e+04 1.26676763e+04
 4.53645596e+03 2.12371484e+03 2.09116797e+03 2.82743100e+03
 3.54672945e+03 4.34745791e+03 5.02654400e+03 1.21500000e+04
 1.38240000e+04 1.56060000e+04 2.40000000e+03 3.21698816e+03
 3.75000000e+03 3.52800000e+03 6.14400000e+03 7.35000000e+03
 2.12176224e+03 2.39313760e+03 1.25663600e+03 8.04247040e+02
 5.83200000e+03 1.60459851e+03 4.02638741e+03 6.82215118e+03
 1.74736824e+03 1.74736824e+03 3.45194737e+03 8.60894024e+03
 2.01061760e+04 2.43284730e+04 3.01718304e+04 5.14718106e+04
 1.80123641e+04 2.32351996e+04 1.77952224e+03 1.38544119e+03
 1.20687321e+03 9.953813

In [ ]:
def calculate_sphericity(row):
    area = row['area']
    volume = row['volume']

    # Проверяем, что площадь и объем больше нуля
    if area <= 0 or volume <= 0:
        return float('nan')  # Возвращаем NaN для некорректных данных

    radius_sphere = (3 * volume / (4 * np.pi)) ** (1/3)
    surface_area_sphere = 4 * np.pi * radius_sphere ** 2
    sphericity = surface_area_sphere / area

    # Ограничиваем сферичность в пределах от 0 до 1
    sphericity = max(0, min(1, sphericity))

    return sphericity

# Применяем функцию ко всему DataFrame
df['sphericity'] = df.apply(calculate_sphericity, axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   volume             1298 non-null   float64
 1   area               1298 non-null   float64
 2   space_group_core   1298 non-null   float64
 3   space_group_shell  1298 non-null   float64
 4   h_range_max_koe    1298 non-null   float64
 5   Tb                 1298 non-null   float64
 6   TN                 1298 non-null   float64
 7   Tc                 1298 non-null   float64
 8   x                  1298 non-null   float64
 9   y                  1298 non-null   float64
 10  z                  1298 non-null   float64
 11  shape              1298 non-null   float64
 12  temperature_k      1298 non-null   float64
 13  sat_em_g           1298 non-null   float64
 14  coer_oe            1298 non-null   float64
 15  mr (emu/g)         1298 non-null   float64
 16  exc_bias_oe        1298 

In [ ]:
print(df['sphericity'].unique())

[1.         0.67113929 0.67113929 0.51664327 0.06436598 0.40775738
 0.80599598 0.80599598 0.80599598 0.67113929 0.51664327 0.49325101
 0.53995481 0.51664327 0.67113929 0.23272239 0.92055376 0.9598049
 0.80599598 0.80599598 0.55615782 0.21839518 0.84278499 0.24326802
 0.87358071 0.13388663 0.26958259 0.03585747 0.05692019]


In [ ]:
def compute_max_min_ratio(df):
    max_min_ratios = []
    for index, row in df.iterrows():
        values = [row['x'], row['y'], row['z']]
        non_zero_values = [value for value in values if value != 0]
        # Check if there are any non-zero values
        if non_zero_values:
            max_value = max(non_zero_values)
            min_value = min(non_zero_values)
            if min_value != 0:  # Ensure we don't divide by zero
                ratio = max_value / min_value
                max_min_ratios.append(ratio)
            else:
                max_min_ratios.append(0)
        else:
            max_min_ratios.append(0)

    df['max/min'] = max_min_ratios
    return df

# Apply the function
df = compute_max_min_ratio(df)

# Print unique values
print(df['max/min'].unique())

[  1.           1.           1.          50.           1.24675325
   1.45433526   1.75028902   5.61724363   1.           2.35696203
   2.05787234   1.           1.           7.27272727   1.
   1.33985582   1.66110184   1.08173077   1.15164835   8.09090909
   6.125        1.21728545   1.9686       2.17580505   8.
   1.05530833  34.146       14.31        11.2285      13.99188
  14.4         14.7484      20.8912      37.028       30.69628
  27.112       27.32632      3.12689739   1.63289474   3.10945274
   4.62107209   2.69978402   2.81995662   5.29531568   3.12876053
   3.3290653    1.55688623   5.52016985   2.42248062   4.76190476
   1.54569061   1.96896905   2.40847784   2.41779497   2.90360046
   6.80498262   8.66666667   1.33333333  92.90540541  88.55585831
  77.96257796   1.30699088   1.13756614   1.7768595    1.51493291
   5.3          5.03333333  16.66666667   1.           5.83333333
 120.25        60.125     ]


In [ ]:
print(df.insert(0, 'area/volume', 0))

None


In [ ]:
# Найдём отношение площади к объёму
for i, _ in df.iterrows():
    df['area/volume'][i] = df.area[i]/df.volume[i]
print(df['area/volume']) # TODO: добавить тут в датасете площадь на объём и проверить подсчёт объёма и площади

0       0.405405
1       0.428571
2       0.176471
3       0.272727
4       0.250000
          ...   
1293    0.158061
1294    0.235479
1295    0.060000
1296    0.060000
1297    0.060000
Name: area/volume, Length: 1298, dtype: float64


<ipython-input-31-354cfa74501d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['area/volume'][i] = df.area[i]/df.volume[i]
<ipython-input-31-354cfa74501d>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.4054054054054055' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['area/volume'][i] = df.area[i]/df.volume[i]


In [ ]:
df['core'] = core
df['shell'] = shel
df['formula'] = all

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   area/volume        1298 non-null   float64
 1   volume             1298 non-null   float64
 2   area               1298 non-null   float64
 3   space_group_core   1298 non-null   float64
 4   space_group_shell  1298 non-null   float64
 5   h_range_max_koe    1298 non-null   float64
 6   Tb                 1298 non-null   float64
 7   TN                 1298 non-null   float64
 8   Tc                 1298 non-null   float64
 9   x                  1298 non-null   float64
 10  y                  1298 non-null   float64
 11  z                  1298 non-null   float64
 12  shape              1298 non-null   float64
 13  temperature_k      1298 non-null   float64
 14  sat_em_g           1298 non-null   float64
 15  coer_oe            1298 non-null   float64
 16  mr (emu/g)         1298 

In [ ]:
df

,area/volume,volume,area,space_group_core,space_group_shell,h_range_max_koe,Tb,TN,Tc,x,...,exc_dir,ver_shift_emu_g,ver_s_dir,fc_field_t,id,sphericity,max/min,core,shell,formula
0,0.405405,1697.396888,688.133874,186.0,227.0,50.0,300.0,378.0,793.0,7.40,...,0.0,0.00000,1.0,5.0,<built-in function id>,1.0,1.0,ZnO,CoFe2O4,ZnO/CoFe2O4
1,0.428571,1436.753827,615.751640,225.0,227.0,50.0,276.0,293.0,843.0,7.00,...,0.0,0.00000,1.0,5.0,<built-in function id>,1.0,1.0,CoO,CoFe2O4,CoO/CoFe2O4
2,0.176471,20579.508893,3631.678040,227.0,0.0,10.0,150.0,766.8,640.0,17.00,...,1.0,0.00000,1.0,0.0,<built-in function id>,1.0,1.0,Mn0.5Zn0.5Fe2O4,0,Mn0.5Zn0.5Fe2O4
3,0.272727,5575.275053,1520.529560,227.0,0.0,10.0,150.0,766.8,640.0,11.00,...,1.0,0.00000,1.0,0.0,<built-in function id>,1.0,1.0,Mn0.5Zn0.5Fe2O4,0,Mn0.5Zn0.5Fe2O4
4,0.250000,7238.223360,1809.555840,227.0,0.0,10.0,150.0,766.8,640.0,12.00,...,1.0,0.00000,1.0,0.0,<built-in function id>,1.0,1.0,Mn0.5Zn0.5Fe2O4,0,Mn0.5Zn0.5Fe2O4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,0.158061,28640.254098,4526.910553,161.0,0.0,70.0,200.0,640.0,1103.0,18.98,...,0.0,2.10662,1.0,7.0,<built-in function id>,1.0,1.0,BiFeO3,NaN,BiFeO3-SG
1294,0.235479,8661.568143,2039.615732,161.0,0.0,70.0,200.0,640.0,1103.0,12.74,...,0.0,0.03832,1.0,7.0,<built-in function id>,1.0,1.0,BiFeO3,NaN,BiFeO3-SG
1295,0.060000,523598.333333,31415.900000,15.0,225.0,30.0,75.0,490.0,720.0,50.00,...,0.0,0.08100,1.0,0.3,<built-in function id>,1.0,1.0,NiCo2O4,NiO,NiCo2O4/NiO
1296,0.060000,523598.333333,31415.900000,15.0,225.0,30.0,75.0,490.0,720.0,50.00,...,0.0,0.65000,1.0,0.3,<built-in function id>,1.0,1.0,NiCo2O4,NiO,NiCo2O4/NiO


In [ ]:
# Выгрузим данные отдельный датасет
df.to_csv(r'bias_zod_f_nun0508.csv')